In [12]:
from typing import List, Optional

import fire

from llama import Dialog, Llama
import torch.distributed as dist
import torch
import os
from data_casual import output_list_train, input_list_train

In [13]:
from data_casual import instruction_data_point

In [14]:

from llama.model import ModelArgs, Transformer
from llama.tokenizer import ChatFormat, Dialog, Message, Tokenizer

In [15]:
import json
import os
import sys
import time
from pathlib import Path
from typing import List, Optional, Tuple, TypedDict

import torch
import torch.nn.functional as F
from fairscale.nn.model_parallel.initialize import (
    get_model_parallel_rank,
    initialize_model_parallel,
    model_parallel_is_initialized,
)

In [5]:
generator = Llama.build("./", "./tokenizer.model",max_seq_len= 2500,max_batch_size= 4, activation=True, activation_layer=28)

> initializing model parallel with size 1
> initializing ddp with size 1


> initializing pipeline with size 1


c:\Users\Ribbe\Coding\VSC\FU_Berlin\Actual_Work\llama3\llama\generation.py:98: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location=

Loaded in 120.28 seconds


In [6]:
ckpt_dir = "./"
tokenizer_path = "./tokenizer.model"
temperature = 0.6
top_p = 0.9
max_seq_len = 1024
max_batch_size = 4
max_gen_len= generator.model.params.max_seq_len - 1


In [7]:
def sample_top_p(probs, p):
    """
    Perform top-p (nucleus) sampling on a probability distribution.

    Args:
        probs (torch.Tensor): Probability distribution tensor.
        p (float): Probability threshold for top-p sampling.

    Returns:
        torch.Tensor: Sampled token indices.

    Note:
        Top-p sampling selects the smallest set of tokens whose cumulative probability mass
        exceeds the threshold p. The distribution is renormalized based on the selected tokens.
    """
    probs_sort, probs_idx = torch.sort(probs, dim=-1, descending=True)
    probs_sum = torch.cumsum(probs_sort, dim=-1)
    mask = probs_sum - probs_sort > p
    probs_sort[mask] = 0.0
    probs_sort.div_(probs_sort.sum(dim=-1, keepdim=True))
    next_token = torch.multinomial(probs_sort, num_samples=1)
    next_token = torch.gather(probs_idx, -1, next_token)
    return next_token

In [8]:
prompt_tokens = [generator.formatter.encode_dialog_prompt(input_list_train[1142])]

In [9]:
n = len(input_list_train)
n

205734

In [10]:
value, activ = generator.chat_completion(instruction_data_point,
                                               max_gen_len=max_gen_len,
                                               top_p=top_p,
                                               temperature=temperature)

[tensor([-0.2812,  0.1436,  0.1602,  ..., -0.1445,  0.0107,  0.0918]),
 tensor([ 0.3418,  0.7930, -0.4766,  ...,  0.0840, -0.0215, -0.5859])]

In [13]:
n=10
for i in range(len(generator.model.layers)):
    instruction_vector = torch.tensor(4096*[0.0])
    generator.change_activation_layer(i)
    print(instruction_vector)
    for k in range(n):
        value_instruct, activation_instruct = generator.chat_completion(instruction_data_point,
                                               max_gen_len=max_gen_len,
                                               top_p=top_p,
                                               temperature=temperature)
        instruction_vector += 1/n*activation_instruct[0]    
    print(instruction_vector)
    print(torch.norm(instruction_vector))
    torch.save(instruction_vector, f"./steering_vectors/instruct_vector{i}.pt")


tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([ 0.0006,  0.0027,  0.0006,  ..., -0.0022, -0.0011, -0.0001])
tensor(0.5859)
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([-0.0011,  0.0017, -0.0022,  ..., -0.0014,  0.0004,  0.0023])
tensor(9.0625)
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([-0.0083, -0.0004,  0.0008,  ..., -0.0027, -0.0021,  0.0044])
tensor(9.0625)
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([-0.0095,  0.0020,  0.0025,  ..., -0.0192, -0.0013,  0.0043])
tensor(9.1250)
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([-0.0141,  0.0256,  0.0139,  ...,  0.0056, -0.0088,  0.0170])
tensor(9.1875)
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([-0.0023,  0.0294, -0.0123,  ..., -0.0232, -0.0019,  0.0211])
tensor(9.2500)
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([-0.0303,  0.0466, -0.0175,  ...,  0.0085,  0.0036,  0.0020])
tensor(9.3750)
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([-0.0255,  0.0221, -0.0086,  ..., -0.0058,  0.0067,  0.0048])
tensor(9.3750)
tensor([0., 0., 

In [19]:
n=1
for i in range(len(generator.model.layers)):
    instruction_vector = torch.tensor(4096*[0.0])
    generator.change_activation_layer(i)
    print(instruction_vector)
    for k in range(n):
        value_instruct, activation_instruct = generator.chat_completion([input_list_train[1142]],
                                               max_gen_len=max_gen_len,
                                               top_p=top_p,
                                               temperature=temperature)
        instruction_vector += 1/n*activation_instruct[0]    
    print(instruction_vector)
    print(torch.norm(instruction_vector))

tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([ 0.0015, -0.0022,  0.0019,  ..., -0.0074,  0.0022,  0.0006])
tensor(0.5273)
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([ 0.0063,  0.0036, -0.0003,  ..., -0.0166, -0.0051, -0.0015])
tensor(2.3750)
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([ 1.4038e-03, -6.7139e-04,  9.1791e-06,  ..., -1.5747e-02,
        -6.5002e-03, -9.8267e-03])
tensor(2.5156)
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([-0.0123,  0.0014,  0.0009,  ..., -0.0011, -0.0134, -0.0076])
tensor(2.7031)
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([-0.0114,  0.0176,  0.0003,  ...,  0.0090, -0.0294, -0.0039])
tensor(2.8594)
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([ 0.0184,  0.0038,  0.0011,  ...,  0.0002, -0.0249,  0.0204])
tensor(3.0781)
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([ 0.0109,  0.0096,  0.0024,  ...,  0.0265, -0.0284,  0.0074])
tensor(3.2188)
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([-0.0031,  0.0064, -0.0060,  ...,  0.0192, -0.0045,  0.0039])

In [22]:
generator.change_activation_layer(28)
neg_activation_vector_dic = {}
pos_activation_vector_dic = {}
loss = 0
instruct_vec = torch.load(f"./steering_vectors/instruct_vector{28}.pt")
for i in range(5):
    value, activation_vec_list = generator.chat_completion([input_list_train[i]],
                                       max_gen_len=max_gen_len,
                                       top_p=top_p,
                                       temperature=temperature)
    value_numerical = "1" if value[0]["generation"]["content"] == "T" else 0
    activation_vector = activation_vec_list[0] - instruct_vec
    if value_numerical == output_list_train[i]:
        pos_activation_vector_dic[i] = activation_vec_list[0] 
    else :
        neg_activation_vector_dic[i] = activation_vec_list[0]
        loss += 1/1000


C:\Users\Ribbe\AppData\Local\Temp\ipykernel_78700\3943913927.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  instruct_vec = torch.load(f"./steering_vectors/instruct_vect

In [10]:
pos_vec_sum = torch.zeros([len(neg_activation_vector_dic[1])])
for ind, item in pos_activation_vector_dic.items():
    pos_vec_sum += 1/len(pos_activation_vector_dic)*item


neg_vec_sum = torch.zeros([len(neg_activation_vector_dic[1])])
for ind, item in neg_activation_vector_dic.items():
    neg_vec_sum += 1/len(neg_activation_vector_dic)*item

steering_vector = pos_vec_sum-neg_vec_sum


In [11]:
print(steering_vector)

tensor([-0.2949,  0.0142, -0.0035,  ..., -0.1094,  0.0864,  0.1484])


In [12]:
torch.save(steering_vector, "./steering_vector_layer28.pt")

In [1]:
import torch

In [28]:
vec = torch.load("./steering_without_instruct/vector24.pt")

C:\Users\Ribbe\AppData\Local\Temp\ipykernel_81916\3186958420.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vec = torch.load("./steering_without_instruct/vector24.pt")


In [19]:
torch.norm(vec)

tensor(7.0938, device='cuda:0', dtype=torch.bfloat16)

In [20]:
vec = torch.nn.functional.normalize(vec, dim=0)

In [23]:
torch.norm(vec)

tensor(1., device='cuda:0', dtype=torch.bfloat16)

In [29]:
torch.mean(vec)

tensor(-0.0008, device='cuda:0', dtype=torch.bfloat16)

In [ ]:
for k in range(32):
    vec = 